In [4]:
!pwd

%cd drive/MyDrive
%cd U-Net




/content
/content/drive/MyDrive
/content/drive/MyDrive/U-Net


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!git clone https://github.com/milesial/Pytorch-UNet.git

Cloning into 'Pytorch-UNet'...
remote: Enumerating objects: 618, done.
remote: Total 618 (delta 0), reused 0 (delta 0), pack-reused 618 (from 1)
Receiving objects: 100% (618/618), 47.42 MiB | 10.76 MiB/s, done.
Resolving deltas: 100% (338/338), done.


In [5]:
%cd Pytorch-UNet

/content/drive/MyDrive/U-Net/Pytorch-UNet


In [6]:
!pip install -r requirements.txt

In [2]:
!pwd
%cd /content/drive/MyDrive/U-Net/Pytorch-UNet

/content
/content/drive/MyDrive/U-Net/Pytorch-UNet


In [5]:
#!python train.py --epochs 5 --batch-size 4 --learning-rate 0.001 --scale 0.5 --validation 10 --amp

usage: train.py [-h] [--epochs E] [--batch-size B] [--learning-rate LR] [--load LOAD]
                [--scale SCALE] [--validation VAL] [--amp] [--bilinear] [--classes CLASSES]
train.py: error: unrecognized arguments: --img-dir /content/drive/MyDrive/U-Net/Pytorch-UNet/data/imgs --mask-dir /content/drive/MyDrive/U-Net/Pytorch-UNet/data/masks


In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from torch import nn
from utils.data_loading import BasicDataset
from unet import UNet


images_dir = '/content/drive/MyDrive/U-Net/Pytorch-UNet/data/imgs'
masks_dir = '/content/drive/MyDrive/U-Net/Pytorch-UNet/data/masks'

# 모델 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet(n_channels=1, n_classes=1)
model.to(device)

# 데이터셋 로드 및 DataLoader 설정
dataset = BasicDataset(images_dir=images_dir, mask_dir=masks_dir, scale=1.0)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# 손실 함수와 옵티마이저 설정
criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 함수 정의
def train(model, dataloader, criterion, optimizer, num_epochs=5):
    model.train()  # 학습 모드로 전환
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in dataloader:
            images = batch['image'].to(device=device, dtype=torch.float32, memory_format=torch.channels_last)  # 이미지를 GPU/CPU에 로드
            masks = batch['mask'].to(device=device, dtype=torch.long)    # 마스크를 GPU/CPU에 로드



            # Forward pass
            outputs = model(images)

            loss = criterion(outputs, masks.float())


            # Backward pass 및 최적화
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # 에포크마다 평균 손실 출력
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader)}")

# 모델 학습
train(model, train_loader, criterion, optimizer, num_epochs=10)

# 학습된 모델 저장
torch.save(model.state_dict(), '/content/drive/MyDrive/U-Net/MODEL.pth')

100%|██████████| 30/30 [00:00<00:00, 78.21it/s]
